In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec

from dekef.base_density import *

from IFdensity.contam_sm_de import *
from IFdensity.influence_function import *

from IPython.display import Markdown as md

In [ ]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFdensity')
true_data = np.load('data/geyser.npy').astype(np.float64)
df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)

# original data with 108.0 removed 
data_waiting = df[df != 108.0]

# bandwidth parameter in the Gaussian kernel function
bw = 7.0

# penalty parameter 
log_pen_param = -10.0

# base density 
base_density = BasedenGamma(np.load('data/geyser.npy').astype(np.float64)[:, 0])

plot_kwargs = plot_IF_1d_params(x_limit=(21., 410.), plot_pts_cnt = 2000)

In [ ]:
# contaminated data 
contam_data_list = np.arange(50., 510., 10)
bw_list = [5., 7., 9.]
log_pen_param_list = [-8., -10., -12.]
if_norm = {}
if_limit_norm = {}

for bw in bw_list: 
    for log_pen_param in log_pen_param_list: 
        
        print('=' * 50)
        print(f'bw={bw}, lambda={log_pen_param}')
        
        if_norm[f'bw={bw}, lambda={log_pen_param}'] = []
        if_limit_norm[f'bw={bw}, lambda={log_pen_param}'] = []
        
        for contam_pt in contam_data_list: 
            
            if contam_pt % 50 == 0: 
                print('-' * 50)
                print(f'Current contaminated data point is {contam_pt}.')

            contam_data = np.array([[contam_pt]])

            ifun = SMInfluenceFunction(
                data = data_waiting, 
                contam_data = contam_data, 
                contam_weight = 1e-8, 
                penalty_param = np.exp(log_pen_param), 
                base_density = base_density, 
                bw = bw)

            if_norm[f'bw={bw}, lambda={log_pen_param}'].append(ifun.eval_IF_natparam_norm())

            if_limit_norm[f'bw={bw}, lambda={log_pen_param}'].append(ifun.eval_IF_natparam_limit_norm_1d())


In [ ]:
plt.plot(contam_data_list, if_norm, 'b-')
plt.plot(contam_data_list, if_limit_norm, 'r-')

In [ ]:
import pandas as pd 

In [ ]:
?pd.concat

In [ ]:
output_pd = pd.DataFrame()
for bw in bw_list: 
    for log_pen_param in log_pen_param_list: 
        
        sub_df = {}
        
        sub_df['bw'] = [bw] * len(contam_data_list)
        sub_df['loglambda'] = [log_pen_param] * len(contam_data_list)
        sub_df['contam_data'] = contam_data_list
        sub_df['IFnorm'] = if_norm[f'bw={bw}, lambda={log_pen_param}']
        sub_df['IFnorm_lim'] = if_limit_norm[f'bw={bw}, lambda={log_pen_param}']
        
        output_pd = pd.concat([output_pd, pd.DataFrame(sub_df)])
        

In [ ]:
output_pd

In [ ]:
bw = 5.
logpen = -8.
subdf = output_pd[(output_pd.bw == bw) & (output_pd.loglambda == logpen)]
plt.figure(figsize = (10, 10))
plt.plot(subdf.contam_data, subdf.IFnorm, color = 'tab:blue', linestyle = 'solid', linewidth = 2)
plt.plot(subdf.contam_data, subdf.IFnorm_lim, color = 'tab:red', linestyle = 'dashed')
plt.xlabel('contaminated observation', fontsize = 15)
plt.ylabel('IF norm', fontsize = 15)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(f'plots/IFnorm_natparam-bw={bw}-loglambda={logpen}.pdf')

In [ ]:


fig = plt.figure(constrained_layout=False)
fig.set_figheight(30)
fig.set_figwidth(30)
ncols = 3
nrows = 3

label_fontsize = 20
info_fontsize = 25
tick_fontsize = 18
linewidth = 3.0
x_label = 'contaminated observation'
y_label = 'IF norm'

spec = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure=fig)

for i in range(nrows): 
    for j in range(ncols): 
        
        ax = fig.add_subplot(spec[i, j])
        bw, log_pen_param = bw_list[i], log_pen_param_list[j]
        
        subdf = output_pd[(output_pd.bw == bw_list[i]) & (output_pd.loglambda == log_pen_param_list[j])]
        ax.plot(subdf.contam_data, subdf.IFnorm, color = 'tab:blue', linestyle = 'solid', linewidth = 2)
        ax.plot(subdf.contam_data, subdf.IFnorm_lim, color = 'tab:red', linestyle = 'dashed')
    
        if i == nrows - 1: 
            ax.set_xlabel(x_label, fontsize = label_fontsize)
        else: 
            ax.set_xticks([])
        if j == 0: 
            ax.set_ylabel(y_label, fontsize = label_fontsize)
        
        ax.tick_params(axis = 'both', labelsize = tick_fontsize)

        info = r"$\sigma$={bw}, $\lambda$=exp({pen})".format(bw = bw_list[i], pen = log_pen_param_list[j])
        ax.text(0.985, 0.055,
                info,
                fontsize = info_fontsize,
                multialignment = 'left',
                horizontalalignment = 'right',
                verticalalignment = 'top',
                transform = ax.transAxes,
                bbox = {'facecolor': 'none',
                        'boxstyle': 'Round, pad=0.2'})

plt.tight_layout()
plt.savefig('plots/geyser-waiting-natparam-IF-norm-limit.pdf')